# 🤗 Transformers Exploration

## Generate some text

Link: [GPT-2 Model Card](https://huggingface.co/openai-community/gpt2)

In [1]:
from transformers import pipeline, set_seed

In [2]:
generator = pipeline('text-generation', model='gpt2')
set_seed(42)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use mps:0


In [3]:
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm a language model, and my project will get better with time, but I think there are a lot more things that can help you"},
 {'generated_text': "Hello, I'm a language model, not a language model, so if I don't have a problem, I can fix it by creating new words"},
 {'generated_text': "Hello, I'm a language model, and I'm trying to learn some stuff. I'll try to do some basic programming and just learn better ways"},
 {'generated_text': "Hello, I'm a language model, but I don't believe in grammar. This will work for every language model. You can define it very quickly"},
 {'generated_text': 'Hello, I\'m a language model, a model of how things should be, and then we look at different things as well." I\'d like to'}]

## Visualise the state dict

In [4]:
from transformers import GPT2LMHeadModel

# Load the smallest GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Access the state dictionary
state_dict = model.state_dict()

# Inspect the state dictionary (e.g., print keys or specific weights)
print("State Dictionary Keys:")
for key, val in state_dict.items():
    print(key, val.shape)

State Dictionary Keys:
transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias torch.Size([2304])
transformer.h.1.attn.c_proj.weight torch.Size([768, 768])
transforme

## Now lets create the GPT class

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [191]:
from dataclasses import dataclass
@dataclass
class GPTConfig:
    block_size: int = 1024 # max sequence length
    vocab_size: int = 50257 # number of tokens: 50,000 BPE merges + 256 bytes tokens + 1 <|endoftext|> token
    n_layers: int = 12 # number of layers
    n_heads: int = 12 # number of heads
    emb_dim: int = 768 # embedding dimension

In [63]:
device= torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
device

device(type='mps')

In [220]:
class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()

        assert config.emb_dim % config.n_heads == 0

        self.c_attn = nn.Linear(config.emb_dim, 3 * config.emb_dim)
        self.c_proj = nn.Linear(config.emb_dim, config.emb_dim)
        self.register_buffer("bias",\
                              torch.tril(torch.ones(config.block_size, config.block_size)).\
                                view(1, 1, config.block_size, config.block_size))
        self.n_heads = config.n_heads
        
    def forward(self, x):
        B, T, C = x.shape

        qkv = self.c_attn(x) # (B, T, 3*C)
        q, k, v = torch.split(qkv, C, dim=2)
        q = q.view(B, T, self.n_heads, C//self.n_heads).transpose(1, 2)
        k = k.view(B, T, self.n_heads, C//self.n_heads).transpose(1, 2)
        v = v.view(B, T, self.n_heads, C//self.n_heads).transpose(1, 2) # (B, nH, T, H)

        attn = q @ k.transpose(-2, -1) * ((k.size(-1)) ** (-0.50)) # (B, nH, T, T)
        attn_mask = attn.masked_fill(self.bias[:, :, :T, :T]==0, float("-inf"))
        attn_scores = torch.softmax(attn_mask, dim=-1)

        out = attn_scores @ v # (B, nH, T, H)
        out = out.transpose(1, 2).contiguous().view(B, T, C)

        out = self.c_proj(out)

        return out

        
class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.emb_dim, 4 * config.emb_dim)
        self.gelu = nn.GELU(approximate="tanh")
        self.c_proj = nn.Linear(4 * config.emb_dim, config.emb_dim)

    def forward(self, x):
        # x shape: (B, T, C)
        x = self.c_fc(x) # (B, T, 4 * C)
        x = self.gelu(x)
        x = self.c_proj(x) # (B, T, C)
        return x


class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.emb_dim)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.emb_dim)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x)) # residual (B,T,C)
        x = x + self.mlp(self.ln_2(x)) # residual (B,T,C)
        return x


class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.emb_dim), #[50257, 768]
            wpe = nn.Embedding(config.block_size, config.emb_dim), #[1024, 768]
            h = nn.ModuleList([Block(config) for _ in range(config.n_layers)]),
            ln_f = nn.LayerNorm(config.emb_dim)
        ))

        self.lm_head = nn.Linear(config.emb_dim, config.vocab_size, bias=False)

        self.block_size = config.block_size

    def forward(self, x, targets=None):
        # x shape (B, T), targets shape (B, T)
        B, T =  x.shape
        token_emb = self.transformer.wte(x) # (B, T, C)
        pos_emb = self.transformer.wpe(torch.arange(0, T,\
                                                     device=device)) # (T, C)
        x = token_emb + pos_emb # (B, T, C)

        for layer in self.transformer.h:
            x = layer(x)

        x = self.transformer.ln_f(x)
        logits = self.lm_head(x) # (B, T, V)

        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, self.vocab_size), targets.view(-1)).item()

        return logits, loss
    
    @classmethod
    def from_pretrained(cls, model_type):
        """Loads pretrained GPT-2 model weights from huggingface"""
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layers=12, n_heads=12, emb_dim=768),  # 124M params
            'gpt2-medium':  dict(n_layers=24, n_heads=16, emb_dim=1024), # 350M params
            'gpt2-large':   dict(n_layers=36, n_heads=20, emb_dim=1280), # 774M params
            'gpt2-xl':      dict(n_layers=48, n_heads=25, emb_dim=1600), # 1558M params
        }[model_type]
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

### Load from pretrained

In [224]:
config = GPTConfig()
config

GPTConfig(block_size=1024, vocab_size=50257, n_layers=12, n_heads=12, emb_dim=768)

In [221]:
model = GPT.from_pretrained(model_type="gpt2")

loading weights from pretrained gpt: gpt2


### Lets sample some text again from tis

In [222]:
model.to(device);
model.eval();

In [225]:
import tiktoken
tokenizer = tiktoken.encoding_for_model("gpt-2")
tokenizer.n_vocab

50257

In [237]:
@torch.no_grad()
def generate(model, text ,max_length, num_return_sequences, k=50):
    # first tokenize the text
    tokens = tokenizer.encode(text)
    tokens = torch.tensor(tokens, device=device).long()
    # next make it (B, T)
    tokens = tokens.unsqueeze(0).expand(num_return_sequences, -1)

    for _ in range(max_length):
        x = tokens[:, -config.block_size:] # (B, T)
        logits, _ = model(x) # (B, T, V)

        logits = logits[:, -1, :] # (B, V)

        probs = F.softmax(logits, dim=-1)

        topk_probs, topk_indices = torch.topk(probs, k, dim=-1)

        ix = torch.multinomial(topk_probs, num_samples=1) # (B, 1)
        sampled_indices = torch.gather(topk_indices, -1, ix) # (B, 1)

        tokens = torch.cat((tokens, sampled_indices), dim=1)

    output_text = tokenizer.decode_batch(tokens.cpu().detach().numpy())
    return output_text

In [238]:
torch.manual_seed(42)
if device == "mps":
    torch.mps.manual_seed(42)
elif device == "cuda":
    torch.cuda.manual_seed(42)

In [239]:
generate(model, "Hello, I'm a language model," ,max_length=30, num_return_sequences=5)

["Hello, I'm a language model, which means I'm familiar with it, but I'm not fluent in that. Well, with that said, I know how to put this together.",
 "Hello, I'm a language model, and the syntax, to make use of it, is pretty good. So why do you have that and not try using something else on it (from",
 "Hello, I'm a language model, I'm doing this work in Python, and then I'm writing code for Haskell.\n\nSo we can see, in this case, my previous",
 "Hello, I'm a language model, and you're making assumptions about my use of them. I'm not a natural language learner. I'm not a linguist. I'm interested",
 "Hello, I'm a language model, well, I'm from Java and have to write a programming language for it. I have my own vocabulary because I think of things differently for the first"]

In [229]:
generator = pipeline('text-generation', model='gpt2')
set_seed(42)

Device set to use mps:0


In [230]:
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm a language model, and my project will get better with time, but I think there are a lot more things that can help you"},
 {'generated_text': "Hello, I'm a language model, not a language model, so if I don't have a problem, I can fix it by creating new words"},
 {'generated_text': "Hello, I'm a language model, and I'm trying to learn some stuff. I'll try to do some basic programming and just learn better ways"},
 {'generated_text': "Hello, I'm a language model, but I don't believe in grammar. This will work for every language model. You can define it very quickly"},
 {'generated_text': 'Hello, I\'m a language model, a model of how things should be, and then we look at different things as well." I\'d like to'}]